In [4]:
import pandas as pd
import numpy as np
import preprocessing
import metrics_experiment
from sklearn.metrics import auc
import metrics_helper
from datetime import timedelta

 
# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [5]:
example1 = pd.read_csv('../tests/test-data/libre-data/3028 2.csv', header=None)
example2 = pd.read_csv('../tests/test-data/libre-data/3028 2.csv')


In [6]:
example3 = pd.read_csv('../tests/test-data/libre-test.csv', header=2)
example3 = example3[['Device Timestamp','Historic Glucose mmol/L']]
example3.columns = ['time', 'glc']
example3 = example3.dropna()

In [44]:
preprocessed_objects = []
for i in [example1, example2]:
    preprocessed_objects.append(preprocessing.preprocess_df(i, '3028 2.csv'))

c:\Users\cr591\Anaconda3\envs\streamlit\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
df = preprocessed_objects[0]['data']

In [27]:
total_df = pd.DataFrame()
for i in preprocessed_objects:
    if i.usable==True:
        df_id = i.data
        df_id['ID'] = i.id
        total_df = total_df.append(i.data)

AttributeError: 'dict' object has no attribute 'usable'

In [5]:
results = total_df.groupby('ID').apply(
            lambda group: metrics_experiment.calculate_all_metrics(group, unit='blo', interval='b'))

blo b
blo b


## LBGI experiment

Kovatchev BP, Cox DJ, Gonder-Frederick LA, Young-Hyman D, Schlundt D, Clarke WL: Assessment of risk for severe hypoglycemia among adults with IDDM: validation of the low blood glucose index. Diabetes Care  21 :1870 –1875, 1998

In [10]:
from datetime import timedelta

In [29]:
df = preprocessed_objects[0]['data']
df.dropna(inplace=True, subset=['glc'])

In [23]:
df.reset_index(drop=True, inplace=True)

In [43]:
df

,time,glc,scan_glc
0,2018-04-20 15:31:00,11.0,NaN
3,2018-04-20 15:46:00,11.5,NaN
4,2018-04-20 16:01:00,11.9,NaN
5,2018-04-20 16:16:00,12.7,NaN
6,2018-04-20 16:31:00,13.1,NaN
...,...,...,...
21818,2017-10-23 18:58:00,2.6,NaN
21819,2017-10-23 19:13:00,2.2,NaN
21821,2017-10-23 19:28:00,2.2,NaN
21822,2017-10-23 19:44:00,2.2,NaN


In [47]:
df.set_index(['time'], inplace=True)

In [48]:
day = df.between_time('06:00','23:59')
night = df[~df.index.isin(day.index)]

In [49]:
day

,glc,scan_glc
time,,
2018-04-20 15:31:00,11.0,NaN
2018-04-20 15:46:00,11.5,NaN
2018-04-20 16:01:00,11.9,NaN
2018-04-20 16:16:00,12.7,NaN
2018-04-20 16:31:00,13.1,NaN
...,...,...
2017-10-23 18:58:00,2.6,NaN
2017-10-23 19:13:00,2.2,NaN
2017-10-23 19:28:00,2.2,NaN


In [50]:
night

,glc,scan_glc
time,,
2018-04-21 00:02:00,2.2,NaN
2018-04-21 00:17:00,2.2,NaN
2018-04-21 00:32:00,2.3,NaN
2018-04-21 00:47:00,2.7,NaN
2018-04-21 01:02:00,3.1,NaN
...,...,...
2017-10-23 04:56:00,2.2,NaN
2017-10-23 05:11:00,2.2,NaN
2017-10-23 05:26:00,2.2,NaN


In [65]:
def calc_bgi(glucose, mmol=True):
    if mmol:
        num1=1.794
        num2=1.026
        num3=1.861
    else:
        num1=1.509
        num2=1.084
        num3=5.381
    bgi = num1*(np.log(glucose)**num2 - num3)
    return bgi
    
def calc_lbgi(glucose, mmol=True):
    bgi = calc_bgi(glucose, mmol)
    lbgi = 10*(min(bgi, 0)**2)
    return lbgi

def calc_hbgi(glucose, mmol=True):
    bgi = calc_bgi(glucose, mmol)
    hbgi = 10*(max(bgi, 0)**2)
    return hbgi

def helper_bgi(df):
    lbgi = df['glc'].apply(lambda x: calc_lbgi(x)).mean()
    hbgi = df['glc'].apply(lambda x: calc_hbgi(x)).mean()
    return {'LBGI': lbgi, 'HBGI':hbgi}

In [70]:
helper_bgi(df)

{'LBGI': 2.979953584229811, 'HBGI': 6.99762682940931}

## Data sufficiency

In [31]:
df = preprocessed_objects[0]['data'].copy()
#df.dropna(subset=['glc', 'time'], inplace=True)

In [32]:
df

,time,glc,scan_glc
0,2018-04-20 15:31:00,11.0,NaN
1,2018-04-20 15:35:00,NaN,10.8
2,2018-04-20 15:35:00,NaN,NaN
3,2018-04-20 15:46:00,11.5,NaN
4,2018-04-20 16:01:00,11.9,NaN
...,...,...,...
21822,2017-10-23 19:44:00,2.2,NaN
21823,2017-10-23 19:50:00,NaN,2.2
21824,2017-10-23 19:58:00,2.2,NaN
21825,2017-10-23 20:06:00,NaN,2.2


In [33]:
test1 = df.iloc[0:10]
#test1.iloc[2]['glc'] = np.nan

In [34]:
test1

,time,glc,scan_glc
0,2018-04-20 15:31:00,11.0,NaN
1,2018-04-20 15:35:00,NaN,10.8
2,2018-04-20 15:35:00,NaN,NaN
3,2018-04-20 15:46:00,11.5,NaN
4,2018-04-20 16:01:00,11.9,NaN
5,2018-04-20 16:16:00,12.7,NaN
6,2018-04-20 16:31:00,13.1,NaN
7,2018-04-20 16:46:00,12.8,NaN
8,2018-04-20 17:01:00,12.8,NaN
9,2018-04-20 17:16:00,13.0,NaN


In [101]:
test3.to_csv('../tests/test-data/perc_missing_test3.csv')

In [36]:
helper_missing(test1, 15)

{'Start Time': '2018-04-20 15:31:00',
 'End Time': '2018-04-20 17:16:00',
 'Data Sufficiency': 100}

In [99]:
test2 = test1.copy()
test2.iloc[1]['glc'] = 10.8
test2.to_csv('../tests/test-data/perc_missing_test2.csv')

c:\Users\cr591\Anaconda3\envs\streamlit\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [97]:
test3 = test1.dropna(subset=['glc'])[['time', 'glc']].reset_index(drop=True)
test3['glc'].iloc[[3,7]]= np.nan


In [98]:
test3

,time,glc
0,2018-04-20 15:31:00,11.0
1,2018-04-20 15:46:00,11.5
2,2018-04-20 16:01:00,11.9
3,2018-04-20 16:16:00,12.7
4,2018-04-20 16:31:00,13.1
5,2018-04-20 16:46:00,12.8
6,2018-04-20 17:01:00,12.8
7,2018-04-20 17:16:00,13.0


,time,glc,scan_glc
3,2018-04-20 15:46:00,11.5,NaN
7,2018-04-20 16:46:00,12.8,NaN


In [92]:
test3['glc'].iloc[[3,7]]= np.nan

c:\Users\cr591\Anaconda3\envs\streamlit\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [93]:
test3

,time,glc,scan_glc
0,2018-04-20 15:31:00,11.0,NaN
1,2018-04-20 15:35:00,NaN,10.8
2,2018-04-20 15:35:00,NaN,NaN
3,2018-04-20 15:46:00,NaN,NaN
4,2018-04-20 16:01:00,11.9,NaN
5,2018-04-20 16:16:00,12.7,NaN
6,2018-04-20 16:31:00,13.1,NaN
7,2018-04-20 16:46:00,NaN,NaN
8,2018-04-20 17:01:00,12.8,NaN
9,2018-04-20 17:16:00,13.0,NaN


In [37]:
test1.dropna(subset=['time', 'glc'])

,time,glc,scan_glc
0,2018-04-20 15:31:00,11.0,NaN
3,2018-04-20 15:46:00,11.5,NaN
4,2018-04-20 16:01:00,11.9,NaN
5,2018-04-20 16:16:00,12.7,NaN
6,2018-04-20 16:31:00,13.1,NaN
7,2018-04-20 16:46:00,12.8,NaN
8,2018-04-20 17:01:00,12.8,NaN
9,2018-04-20 17:16:00,13.0,NaN


In [58]:
((test1.time.iloc[-1] - test1.time.iloc[0])+timedelta(minutes=15))/+timedelta(minutes=15)

8.0

In [39]:
number_readings = sum(test1.set_index('time').groupby(pd.Grouper(freq='15min')).count()['glc'] > 0)


In [40]:
number_readings

8

In [53]:
number_readings = sum(test2.set_index('time').groupby(pd.Grouper(freq='15min')).count()['glc'] > 0)
number_readings

8

In [42]:
start_time = test1['time'].iloc[0]
end_time = test1['time'].iloc[-1]
time_diff = (end_time - start_time)
total_readings = time_diff.total_seconds() / (60 * 15)

In [76]:
test3.iloc[7]['glc'] = np.nan

In [77]:
test3

,time,glc,scan_glc
0,2018-04-20 15:31:00,11.0,NaN
1,2018-04-20 15:35:00,NaN,10.8
2,2018-04-20 15:35:00,NaN,NaN
3,2018-04-20 15:46:00,11.5,NaN
4,2018-04-20 16:01:00,11.9,NaN
5,NaT,NaN,NaN
6,2018-04-20 16:31:00,13.1,NaN
7,NaT,NaN,NaN
8,2018-04-20 17:01:00,12.8,NaN
9,2018-04-20 17:16:00,13.0,NaN


In [72]:
test3.dropna(subset=['glc'])

,time,glc,scan_glc
0,2018-04-20 15:31:00,11.0,NaN
3,2018-04-20 15:46:00,11.5,NaN
4,2018-04-20 16:01:00,11.9,NaN
5,2018-04-20 16:16:00,12.7,NaN
6,2018-04-20 16:31:00,13.1,NaN
7,2018-04-20 16:46:00,12.8,NaN
8,2018-04-20 17:01:00,12.8,NaN
9,2018-04-20 17:16:00,13.0,NaN


In [66]:
helper_missing(test1, 15)

{'Start Time': '2018-04-20 15:31:00',
 'End Time': '2018-04-20 17:16:00',
 'Data Sufficiency': 100}

In [68]:
helper_missing(test2, 15)

{'Start Time': '2018-04-20 15:31:00',
 'End Time': '2018-04-20 17:16:00',
 'Data Sufficiency': 100}

In [94]:
helper_missing(test3, 15)

6


{'Start Time': '2018-04-20 15:31:00',
 'End Time': '2018-04-20 17:16:00',
 'Data Sufficiency': 75.0}

In [73]:
def helper_missing(df, gap_size): #, start_time, end_time
    """
    Helper for percent_missing function
    """
    # Calculate start and end time from dataframe
    start_time = df['time'].iloc[0]
    end_time = df['time'].iloc[-1]
    
    if gap_size == 5:
        freq = '5min'
    elif gap_size==15:
        freq = '15min'
    else:
        return print('EXPLODE THE PROGRAM')
    
    # calculate the number of non-null values
    number_readings = sum(df.set_index('time').groupby(pd.Grouper(freq=freq)).count()['glc'] > 0)
    # calculate the missing data based on start and end of df
    total_readings = ((end_time - start_time)+timedelta(minutes=gap_size))/+timedelta(minutes=gap_size)


    if number_readings >= total_readings:
        data_sufficiency = 100
    else:
        data_sufficiency = number_readings*100/total_readings
    
    return {'Start Time': str(start_time.round('min')), 'End Time':str(end_time.round('min')), 'Data Sufficiency':np.round(data_sufficiency, 1)}


In [10]:
helper_missing(df, 15)

{'Start Time': '2016-01-08 00:01:00',
 'End Time': '2018-12-09 23:54:00',
 'Data Sufficiency': 19.4}

In [5]:

df['time'] = pd.to_datetime(df['time'])
df.sort_values('time', inplace=True)
df.reset_index(drop=True, inplace=True)
df.dropna(subset=['glc', 'time'], inplace=True)

In [39]:
df.time = df.time.apply(lambda x: x.timestamp())

In [42]:
df.describe()

,glc,scan_glc
count,19870.000000,0.0
mean,8.498440,NaN
std,3.667772,NaN
min,2.200000,NaN
25%,5.800000,NaN
50%,8.300000,NaN
75%,11.000000,NaN
max,27.800000,NaN


In [92]:
df.time

0       2018-04-20 15:31:00
3       2018-04-20 15:46:00
4       2018-04-20 16:01:00
5       2018-04-20 16:16:00
6       2018-04-20 16:31:00
                ...        
21818   2017-10-23 18:58:00
21819   2017-10-23 19:13:00
21821   2017-10-23 19:28:00
21822   2017-10-23 19:44:00
21824   2017-10-23 19:58:00
Name: time, Length: 19870, dtype: datetime64[ns]

In [31]:
df.time.values.astype(float)

array([1.45221126e+09, 1.45221216e+09, 1.45221306e+09, ...,
       1.54439784e+09, 1.54439874e+09, 1.54439964e+09])

In [37]:
df

,time,glc,scan_glc
0,1970-01-01 00:00:01.452211260,3.6,NaN
1,1970-01-01 00:00:01.452212160,3.1,NaN
2,1970-01-01 00:00:01.452213060,2.3,NaN
3,1970-01-01 00:00:01.452213960,2.2,NaN
4,1970-01-01 00:00:01.452214860,2.2,NaN
...,...,...,...
21822,1970-01-01 00:00:01.544396340,NaN,NaN
21823,1970-01-01 00:00:01.544396940,4.9,NaN
21824,1970-01-01 00:00:01.544397840,5.7,NaN
21825,1970-01-01 00:00:01.544398740,5.2,NaN


## AUC

In [5]:
df = preprocessed_objects[0]['data']
df = df.dropna(subset=['time', 'glc']).sort_values('time').reset_index(drop=True)

NameError: name 'preprocessed_objects' is not defined

In [7]:
df = example3
df.time = pd.to_datetime(df.time)

In [8]:
def calculate_auc(df):
    if df.shape[0]>1:
        start_time = df.time.iloc[0]
        mins_from_start = df.time.apply(lambda x: x-start_time)
        df['hours_from_start'] = mins_from_start.apply(lambda x: (x.total_seconds()/60)/60)
        avg_auc = auc(df['hours_from_start'], df['glc'])#/24
        return avg_auc
    else:
        return np.nan

In [9]:
def calculate_auc(df):
    hourly_breakdown = df.groupby([df['time'].dt.date, df['time'].dt.hour]).apply(lambda group: calculate_auc(group))#.reset_index()
    
    # Hacking the weird groupby index labelling
    hourly_breakdown = hourly_breakdown.reset_index(level=1)
    hourly_breakdown.columns = ['hour', 'auc']
    hourly_breakdown = hourly_breakdown.reset_index()
    hourly_breakdown.columns = ['date', 'hour', 'auc']
    daily_breakdown = hourly_breakdown.groupby('date').auc.mean()

    hourly_avg = hourly_breakdown.auc.mean()
    #daily_auc = df.groupby(df['time'].dt.date).apply(lambda group: calculate_auc(group))/24 # .reset_index()
    #daily_avg = daily_auc.mean()
    return hourly_avg, daily_breakdown, hourly_breakdown 

In [10]:
hourly_avg,  hourly_breakdown = calculate_auc(df)

: 

: 

In [1]:
datimeshiz = df.groupby([df['time'].dt.date, df['time'].dt.hour]).apply(lambda group: calculate_daily_auc(group))#.reset_index()

NameError: name 'df' is not defined

In [48]:
datimeshiz = datimeshiz.reset_index(level=0)
datimeshiz.columns = ['date', 'auc']
datimeshiz = datimeshiz.reset_index(level=0)


In [49]:
datimeshiz

,time,date,auc
0,0,2020-01-13,4.1250
1,1,2020-01-13,4.2000
2,2,2020-01-13,4.5500
3,3,2020-01-13,4.4625
4,4,2020-01-13,3.9125
5,5,2020-01-13,3.8375
6,6,2020-01-13,3.5375
7,7,2020-01-13,3.6500
8,8,2020-01-13,3.2875
9,9,2020-01-13,3.2750


In [25]:
hourly_auc = df.groupby(df['time'].dt.hour).apply(lambda group: calculate_daily_auc(group)).reset_index()


ValueError: x is neither increasing nor decreasing : [ 0.00000e+00  2.50000e-01  5.00000e-01  7.50000e-01  7.20000e+02
  7.20250e+02  7.20500e+02  7.20750e+02 -7.03200e+03 -7.03175e+03
 -7.03150e+03 -7.03125e+03 -7.00800e+03 -7.00775e+03 -7.00750e+03
 -7.00725e+03 -6.98400e+03 -6.98375e+03 -6.98350e+03 -6.98325e+03
 -6.96000e+03 -6.95975e+03 -6.95950e+03 -6.95925e+03 -6.93600e+03
 -6.93575e+03 -6.93550e+03 -6.93525e+03 -6.91200e+03 -6.91175e+03
 -6.91150e+03 -6.91125e+03 -6.88800e+03 -6.88775e+03 -6.88750e+03
 -6.88725e+03 -6.86400e+03 -6.86375e+03 -6.86350e+03 -6.86325e+03
 -6.84000e+03 -6.83975e+03 -6.83950e+03 -6.83925e+03 -6.81600e+03
 -6.81575e+03 -6.81550e+03 -6.81525e+03 -6.79200e+03 -6.79175e+03
 -6.79150e+03 -6.79125e+03 -6.76800e+03 -6.76775e+03 -6.76750e+03
 -6.76725e+03].

In [23]:
dailyauc = calculate_auc(df)

In [24]:
dailyauc

,time,0
0,2020-01-13,4.209375
1,2020-01-14,3.804688
2,2020-01-15,4.095833
3,2020-01-16,4.269792
4,2020-01-17,4.633333
5,2020-01-18,4.391146
6,2020-01-19,4.356250
7,2020-01-20,4.784896
8,2020-01-21,4.682292
9,2020-01-22,4.253646


In [24]:
newdf

,time,glc,scan_glc,diff,mins_from_start
12675,2016-01-08 00:01:00,3.6,NaN,-206 days +00:15:00,0 days 00:00:00
12676,2016-01-08 00:16:00,3.1,NaN,0 days 00:15:00,0 days 00:15:00
12677,2016-01-08 00:31:00,2.3,NaN,0 days 00:15:00,0 days 00:30:00
12678,2016-01-08 00:46:00,2.2,NaN,0 days 00:15:00,0 days 00:45:00
12679,2016-01-08 01:01:00,2.2,NaN,0 days 00:15:00,0 days 01:00:00
...,...,...,...,...,...
11393,2018-12-09 22:54:00,3.1,NaN,0 days 00:14:00,1066 days 22:53:00
11396,2018-12-09 23:09:00,4.9,NaN,0 days 00:15:00,1066 days 23:08:00
11397,2018-12-09 23:24:00,5.7,NaN,0 days 00:15:00,1066 days 23:23:00
11398,2018-12-09 23:39:00,5.2,NaN,0 days 00:15:00,1066 days 23:38:00


In [30]:
newdf['mins_from_start'].apply(lambda x: (x.total_seconds()/60)/60)

12675        0.000000
12676        0.250000
12677        0.500000
12678        0.750000
12679        1.000000
             ...     
11393    25606.883333
11396    25607.133333
11397    25607.383333
11398    25607.633333
11399    25607.883333
Name: mins_from_start, Length: 19870, dtype: float64

time
2016-01-08    {'ID': '1', 'Average glucose': 5.0645833333333...
2016-01-10    {'ID': '1', 'Average glucose': 2.6545454545454...
2016-02-08    {'ID': '1', 'Average glucose': 6.3168421052631...
2016-02-10    {'ID': '1', 'Average glucose': 9.4000000000000...
2016-03-08    {'ID': '1', 'Average glucose': 8.2263157894736...
                                    ...                        
2018-11-09    {'ID': '1', 'Average glucose': 9.7083333333333...
2018-12-06    {'ID': '1', 'Average glucose': 7.0347826086956...
2018-12-07    {'ID': '1', 'Average glucose': 5.6645833333333...
2018-12-08    {'ID': '1', 'Average glucose': 8.9927083333333...
2018-12-09    {'ID': '1', 'Average glucose': 8.8989583333333...
Length: 236, dtype: object

In [46]:
auc(df.time.values.astype(float), df.glc.values)

714733344.0

In [45]:
auc(df.time, df.glc)

UFuncTypeError: Cannot cast ufunc 'less' input 0 from dtype('<m8[ns]') to dtype('<m8') with casting rule 'same_kind'

In [4]:
dict_results = []
for i in preprocessed_objects:
    if i.usable==True:
        df_id = i.data
        results = metrics_experiment.calculate_all_metrics(df_id, ID=i.id, unit=i.units, interval=i.interval)
        dict_results.append(results)

In [5]:
pd.DataFrame.from_dict(dict_results)

,ID,Average glucose,SD,CV,eA1c,TIR normal,TIR hypoglycaemia,tir_lv1_hypo,tir_lv2_hypo,tir_hyper,tir_lv1_hyper,tir_lv2_hyper,total_number_hypos,number_lv1_hypos,number_lv2_hypos,avg_length_of_hypo,total_time_in_hypos
0,3028,8.49844,3.667772,43.158179,6.973862,55.81,11.11,4.98,6.13,33.07,25.37,7.71,308,131,177,0 days 14:04:17,180 days 13:59:00
1,3028 2,8.49844,3.667772,43.158179,6.973862,55.81,11.11,4.98,6.13,33.07,25.37,7.71,308,131,177,0 days 14:04:17,180 days 13:59:00


In [8]:
pd.DataFrame(dict_results)

,avg,sd,cv,id
0,8.49844,3.667772,43.158179,3028
1,8.49844,3.667772,43.158179,3028 2


In [7]:
metrics.time_in_range(total_df)

,ID,tir_norm,tir_hypo,tir_lv1_hypo,tir_lv2_hypo,tir_hyper,tir_lv1_hyper,tir_lv2_hyper
0,3028,55.81,11.11,4.98,6.13,33.07,25.37,7.71
1,3028 2,55.81,11.11,4.98,6.13,33.07,25.37,7.71


In [13]:
result_metrics = pd.DataFrame()
for i in preprocessed_objects:
    if i.usable==True:
        tir = metrics.time_in_range(i.data)
        tir['ID'] = i.id
    else:
        #log something


NameError: name 'metrics' is not defined

In [12]:
total_df

,time,glc,scan_glc,ID
0,2018-04-20 15:31:00,11.0,NaN,3028
1,2018-04-20 15:35:00,NaN,10.8,3028
2,2018-04-20 15:35:00,NaN,NaN,3028
3,2018-04-20 15:46:00,11.5,NaN,3028
4,2018-04-20 16:01:00,11.9,NaN,3028
...,...,...,...,...
21820,2017-10-23 18:58:00,2.6,NaN,3028 2
21821,2017-10-23 19:13:00,2.2,NaN,3028 2
21823,2017-10-23 19:28:00,2.2,NaN,3028 2
21824,2017-10-23 19:44:00,2.2,NaN,3028 2


In [ ]:
person1 = checkDevices.checkDevices(example1)

hello


c:\Users\cr591\Anaconda3\envs\streamlit\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [3]:
person2 = checkDevices.checkDevices(example2)

hello


In [4]:
uk_data = person2.data
us_data = person2.data

In [5]:
us_data['glc'] = us_data['glc']/0.0557

TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [4]:
person1.data

,time,glc,scan_glc
0,20-04-2018 15:31,11.0,NaN
1,20-04-2018 15:35,NaN,10.8
2,20-04-2018 15:35,NaN,NaN
3,20-04-2018 15:46,11.5,NaN
4,20-04-2018 16:01,11.9,NaN
...,...,...,...
21822,23-10-2017 19:44,2.2,NaN
21823,23-10-2017 19:50,NaN,2.2
21824,23-10-2017 19:58,2.2,NaN
21825,23-10-2017 20:06,NaN,2.2


In [30]:
if pd.notnull(example1.iloc[0,2]):
    print('booo')

In [27]:
if example1.iloc[0,2]:
    print('booo')

booo


In [10]:
header =['Meter',	'Serial Number',	'Meter Timestamp',	'Record Type',	'Historic Glucose(mmol/L)',	'Scan Glucose(mmol/L)']


In [16]:
print(set(example1.iloc[1]))

{nan, 'Scan Glucose(mmol/L)', 'Correction Insulin (units)', 'Historic Glucose(mmol/L)', 'Rapid-Acting Insulin (units)', 'Non-numeric Rapid-Acting Insulin', 'Long-Acting Insulin (units)', 'Serial Number', 'Notes', 'Carbohydrates (servings)', 'Ketone(mmol/L)', 'Record Type', 'Carbohydrates (grams)', 'Non-numeric Long-Acting Insulin', 'Non-numeric Food', 'Meter', 'User Change Insulin (units)', 'Strip Glucose(mmol/L)', 'Meter Timestamp', 'Meal Insulin (units)'}


In [17]:
if set(header).issubset(set(example1.iloc[1])):
    print('booo')

booobs


In [5]:
example1.count().max()

21829

In [3]:
preprocessing.preprocess_data(example1)

{'dt': ['14-09-2018'], 'glc_uk': ['Robert Andrews'], 'glc_us': []}


,time,glc
0,20-04-2018 15:31,11.0
1,20-04-2018 15:46,11.5
2,20-04-2018 16:01,11.9
3,20-04-2018 16:16,12.7
4,20-04-2018 16:31,13.1
...,...,...
19865,23-10-2017 18:58,2.6
19866,23-10-2017 19:13,2.2
19867,23-10-2017 19:28,2.2
19868,23-10-2017 19:44,2.2
